<a href="https://colab.research.google.com/github/nhso/eng-nsho-scraping/blob/Version-0.2-Update-fine-pdf-URLs/%E0%B8%AA%E0%B8%9B%E0%B8%AA%E0%B8%8A_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from pdfcrawler import pdf_to_text_url

In [4]:
url = "http://eng.nhso.go.th/view/1/News/EN-US/?page="
page = 1
new_urls = []

while page <= 10 :
  data = requests.get(url + str(page))
  soup = BeautifulSoup(data.text,'html.parser')
  page += 1
  for i in range(1, 12) :
    ne = soup.find_all("a", {'class' : 'title-7'})[i]
    new_urls.append(ne)

url = "http://eng.nhso.go.th/view/1/In_Depth/EN-US/?page="
page = 1
In_Depth_urls = []

while page <= 11 :
  data = requests.get(url + str(page))
  soup = BeautifulSoup(data.text,'html.parser')
  page += 1
  for i in range(1, 8) :
    IDep = soup.find_all("a", {'href' : 'javascript:;'})[i]
    new_urls.append(IDep)
    
string_list = [str(element) for element in new_urls]
new_urls = pd.DataFrame(string_list, columns=['Raw'])

In [5]:
df_new = new_urls['Raw'].str.split("'", expand=True)
df_new.columns = ['Column1','URLs','Column1','Column1','Column1','Column1','Column1','Column1','Column1','Column1','Column1','Column1','Column1','Column1','Column1']
columns_drop = ['Column1']
news_urls = df_new.drop(columns_drop, axis=1)
news_urls['URLs'] = news_urls['URLs'].str.replace('https://', 'http://', case=False)
news_urls_list = news_urls['URLs'].tolist()

In [6]:
news_data = []
news_name = []

for i in news_urls_list :
  data = requests.get(i)
  soup = BeautifulSoup(data.text)
  for j in soup.find_all("div", {"class" : "post_desc"}) :
    news_data.append(j.text)
  for k in soup.find_all("div", {"class" : "title pt-40"}) :
    news_name.append(k.text)  

In [9]:
news_table = pd.DataFrame([news_name, news_data]).transpose()
news_table.columns = ['Title_Date', 'DATA']
news_table = news_table.replace(r'\n', ' ', regex=True)
news_table = news_table.replace(r'\r', ' ', regex=True)
news_table

,Title_Date,DATA
0,Community Health Fund supports free glasses t...,Community Health Fund supports free glasses ...
1,​Acellular pertussis vaccines offered to preg...,Acellular pertussis vaccines offered to pre...
2,Patients are satisfied with 'Cancer Anywhere'...,Patients are satisfied with 'Cancer Anywhere...
3,Nursery and midwifery clinics improve primary...,Nursery and midwifery clinics improve prima...
4,National Health Security Office of Thailand C...,National Health Security Office of T...
...,...,...
182,Thai Strong Healthcare scheme makes the pande...,An exclusive interview with WHO Representativ...
183,Develop an effective audit system for UCS M...,"By Karoon Kuntiranont, Deputy Secretary-Gener..."
184,Develop an effective audit system for UCS M...,"By Karoon Kuntiranont, Deputy Secretary-Gener..."
185,Thailand to share knowhow of DRG payment syst...,An exclusive interview with NHSO deputy secre...


In [10]:
url = ["http://eng.nhso.go.th/view/1/Annual_Reports/EN-US","http://eng.nhso.go.th/view/1/UHC_Public_Relations/EN-US","http://eng.nhso.go.th/view/1/Articles/EN-US","http://eng.nhso.go.th/view/1/Books/EN-US","http://eng.nhso.go.th/view/1/Leaflets/EN-US", "http://eng.nhso.go.th/view/1/Books/EN-US/?page=2", "http://eng.nhso.go.th/view/1/Books/EN-US/?page=3", "http://eng.nhso.go.th/view/1/Books/EN-US/?page=4", "http://eng.nhso.go.th/view/1/Books/EN-US/?page=5", "http://eng.nhso.go.th/view/1/Reports/EN-US", "http://eng.nhso.go.th/view/1/Reports/EN-US/?page=2", "http://eng.nhso.go.th/view/1/Companion_Book/EN-US"]

pdf_url = []
pdf_name = []
for i in url:
  data = requests.get(i)
  soup = BeautifulSoup(data.text)
  for j in soup.find_all("a",style="text-align:left;",href="javascript:;") :
    pdf_name.append(j.text)
    pdf_url.append(j)

table = pd.DataFrame([pdf_name, pdf_url]).transpose()
table.columns = ['Title', 'DATA']
table['DATA'] = table['DATA'].astype(str)

In [11]:
df_new = table['DATA'].str.split("'", expand=True)
df_new.columns = ['Column1','URLs','Column1','Column1','Column1','Column1','Column1','Column1','Column1','Column1']
columns_drop = ['Column1']
pdf_only_urls = df_new.drop(columns_drop, axis=1)
pdf_only_urls['URLs'] = 'http://eng.nhso.go.th' + pdf_only_urls['URLs']
pdf_only_urls = pdf_only_urls.reset_index().drop(columns='index')
pdftitle = table['Title']
pdf_table = pd.concat([pdftitle, pdf_only_urls], axis=1)
pdf_table = pdf_table.drop(pdf_table[pdf_table['URLs'] == 'http://eng.nhso.go.th/assets/portals/1/files/Thailand_UHC/64-2_P&amp;P_Book(Eng).pdf'].index)
pdf_table = pdf_table.drop(pdf_table[pdf_table['URLs'] == 'http://eng.nhso.go.th/assets/portals/1/files/other/02 The  PD First Policy in Thailand.pdf'].index)
pdf_table = pdf_table.drop(pdf_table[pdf_table['URLs'] == 'http://eng.nhso.go.th/assets/portals/1/files/other/01 Diffinition benefit of UHC  From Pragmatism to Sophistication .pdf'].index)
pdf_table = pdf_table.drop(pdf_table[pdf_table['URLs'] == 'http://eng.nhso.go.thhttp://stream.nhso.go.th/wp-content/uploads/ebooks/NHSO_BP_Mag8/'].index)
pdf_table = pdf_table.reset_index(drop=True)
pdf_table

,Title,URLs
0,NHSO Annual Report Year 2021,http://eng.nhso.go.th/assets/portals/1/files/n...
1,NHSO Annual Report Year 2020,http://eng.nhso.go.th/assets/portals/1/files/a...
2,NHSO Annual report year 2019,http://eng.nhso.go.th/assets/portals/1/files/a...
3,NHSO Annual report year 2018,http://eng.nhso.go.th/assets/portals/1/files/a...
4,NHSO Annual report year 2017,http://eng.nhso.go.th/assets/portals/1/files/a...
...,...,...
88,PMAC FT2015 Global Health post 2015,http://eng.nhso.go.th/assets/portals/1/files/P...
89,PMAC FT2014 Health work force,http://eng.nhso.go.th/assets/portals/1/files/P...
90,PMAC FT2013 One Health,http://eng.nhso.go.th/assets/portals/1/files/P...
91,PMAC FT2012 UHC,http://eng.nhso.go.th/assets/portals/1/files/P...


In [12]:
pdf_body = []

for i in pdf_table['URLs'] :
    pdf_body.append(pdf_to_text_url(i))
    
pdf_table['Data'] = pdf_body

pdf_table

,Title,URLs,Data
0,NHSO Annual Report Year 2021,http://eng.nhso.go.th/assets/portals/1/files/n...,117711NHSO\nAnnual Report\nFiscal Year 2021A m...
1,NHSO Annual Report Year 2020,http://eng.nhso.go.th/assets/portals/1/files/a...,NHSO\nAnnual Report\n2020A message from\nthe C...
2,NHSO Annual report year 2019,http://eng.nhso.go.th/assets/portals/1/files/a...,NHSO\nAnnual\nReport\nFiscal Year 2019Costs re...
3,NHSO Annual report year 2018,http://eng.nhso.go.th/assets/portals/1/files/a...,"NHSO\nAnnual Report,\nFiscal Year 2018A Messag..."
4,NHSO Annual report year 2017,http://eng.nhso.go.th/assets/portals/1/files/a...,NHSO\nAnnual Report\nfiscal year\n2017A messag...
...,...,...,...
88,PMAC FT2015 Global Health post 2015,http://eng.nhso.go.th/assets/portals/1/files/P...,SUPPORTED BY AUTHORS\nPrince Mahidol Award Fou...
89,PMAC FT2014 Health work force,http://eng.nhso.go.th/assets/portals/1/files/P...,
90,PMAC FT2013 One Health,http://eng.nhso.go.th/assets/portals/1/files/P...,Clipping the Wings\nof Emerging Diseases\nin T...
91,PMAC FT2012 UHC,http://eng.nhso.go.th/assets/portals/1/files/P...,Universal Health Coverage\nCase studies from T...


In [13]:
pdf_table = pdf_table.replace(r'\n', ' ', regex=True)
pdf_table = pdf_table.replace(r'\r', ' ', regex=True)
pdf_table 

,Title,URLs,Data
0,NHSO Annual Report Year 2021,http://eng.nhso.go.th/assets/portals/1/files/n...,117711NHSO Annual Report Fiscal Year 2021A mes...
1,NHSO Annual Report Year 2020,http://eng.nhso.go.th/assets/portals/1/files/a...,NHSO Annual Report 2020A message from the Chai...
2,NHSO Annual report year 2019,http://eng.nhso.go.th/assets/portals/1/files/a...,NHSO Annual Report Fiscal Year 2019Costs reduc...
3,NHSO Annual report year 2018,http://eng.nhso.go.th/assets/portals/1/files/a...,"NHSO Annual Report, Fiscal Year 2018A Message ..."
4,NHSO Annual report year 2017,http://eng.nhso.go.th/assets/portals/1/files/a...,NHSO Annual Report fiscal year 2017A message f...
...,...,...,...
88,PMAC FT2015 Global Health post 2015,http://eng.nhso.go.th/assets/portals/1/files/P...,SUPPORTED BY AUTHORS Prince Mahidol Award Foun...
89,PMAC FT2014 Health work force,http://eng.nhso.go.th/assets/portals/1/files/P...,
90,PMAC FT2013 One Health,http://eng.nhso.go.th/assets/portals/1/files/P...,Clipping the Wings of Emerging Diseases in Tha...
91,PMAC FT2012 UHC,http://eng.nhso.go.th/assets/portals/1/files/P...,Universal Health Coverage Case studies from Th...


In [14]:
url = ["http://eng.nhso.go.th/view/1/Philosophy_Background/EN-US", "http://eng.nhso.go.th/view/1/National_Health_Security_Act_B.E.2545/EN-US", "http://eng.nhso.go.th/view/1/Contact_NHSO/EN-US"]

body = []
title_t = []

for i in url:
  data = requests.get(i)
  soup = BeautifulSoup(data.text,)
  for j in soup.find_all("div",{"id":"content-B"}) :
    body.append(j.text)
    title_t.append(j("h1", {"class":"title-head"}))

In [16]:
url = ["http://eng.nhso.go.th/view/1/Vision_Mission/EN-US", "http://eng.nhso.go.th/view/1/NHSO_Board/EN-US", "http://eng.nhso.go.th/view/1/NHSO_Executive/EN-US", "http://eng.nhso.go.th/view/1/NHSB/EN-US", "http://eng.nhso.go.th/view/1/QSC/EN-US"]

for i in url:
  data = requests.get(i)
  soup = BeautifulSoup(data.text,)
  for j in soup.find_all("div",{"id":"content-A"}) :
    body.append(j.text)
    title_t.append(j("h1", {"class":"title-head"}))
    
table_d = pd.DataFrame(body)
table_d.columns = ['DATA']

table_d = table_d.replace(r'\n', ' ', regex=True)
table_d = table_d.replace(r'\r', ' ', regex=True)
table_d


,DATA
0,Philosophy & Background Philosophy “Hea...
1,National Health Security Act B.E.2545 (A.D.2...
2,National Health Security Office Th...
3,HomeAbout_NHSOVision/Mission Vision Ever...
4,HomeAbout_NHSONHSO_StructureNHSO BoardNHSO’...
5,HomeAbout_NHSONHSO_StructureNHSO Executive ...
6,HomeAbout_NHSONHSO_StructureNational Health...
7,HomeAbout_NHSONHSO_StructureQuality and Sta...
8,HomeAbout_NHSOVision/Mission Vision Ever...
9,HomeAbout_NHSONHSO_StructureNHSO BoardNHSO’...


In [23]:
table_d.to_csv("Data/web.csv")
news_table.to_csv("Data/news.csv")
pdf_table.to_csv("Data/pdf.csv")